## Import Dependencies

In [2]:
import pandas as pd
import numpy as np
from pycaret.datasets import get_data
from pycaret.classification import *
import os

## Define functions

In [3]:
dataset_prefix = '0_01-partial-dataset'

In [4]:
def save_pre_processed_dataset(df: pd.DataFrame, name):
    """Save as the dataset pre-processed as csv in the dir root/Datasets/CSE-CIC/IDS2018/pre-processed/"""
    file_name = dataset_prefix + '-' + name + '.csv'
    
    # Define the path to save the dataset pre-processed
    file_path = os.path.join(os.getcwd(), '..', 'Datasets', 'CSE-CIC-IDS2018', 'pre-processed', file_name)

    # Save the dataset pre-processed
    df.to_csv(file_path ,index=False)

In [6]:
def test_and_save_pre_processing_approach(approach_name, setup):
    # Test different algorithms
    setup.compare_models(include = ['ada', 'gbc', 'et','xgboost', 'rf', 'dt', 'lightgbm'])
    # setup.compare_models()
    
    # Getting the resulting df of the models_comparison
    df_models_comparison = pull().copy()
    
    # Getting the dataset_pre_processed by the autoML
    df_dataset_pre_processed = setup.get_config('dataset_transformed').copy()

    # Save as the dataset pre-processed as csv in the dir root/Datasets/CSE-CIC/IDS2018/pre-processed/
    save_pre_processed_dataset(df_dataset_pre_processed, approach_name)

    return df_models_comparison, df_dataset_pre_processed


## Set display unlimited number of lines

In [7]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

## Load datasets in pandas dataframes

In [8]:
# Define the path to the folder containing the CSV files
file_path = os.path.join(os.getcwd(), '..', 'Datasets', 'CSE-CIC-IDS2018', 'raw', '0_01-partial-dataset.csv')
# Import csv to pandas
dataset = pd.read_csv(file_path)

# Pre-processing

## 1. First steps for pycaret be able to consume

#### Replace -inf/+inf for Nan

In [9]:
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

#### Convert Timestamp values to pandas date and time datetime64 format

In [10]:
dataset['Timestamp'] = pd.to_datetime(dataset['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

## 2. Pre-processing using Pycaret

### 2.1 Generic pre-processing specifications techniques
The autoML choose the following parameters automatically:
- inputation_type: simple
    - numeric_imputation: mean
    - categorical_imputation: mode
- fold_strategy: stratifiedkfold
    - fold: 10

In [19]:
generic_pre_processing_setup = setup(
                                dataset, 
                                target = 'Label',
                                imputation_type = 'simple',
                                numeric_imputation = 'mean',
                                categorical_imputation = 'mode',
                                fold_strategy = 'stratifiedkfold',
                                fold = 10
                                )

,Description,Value
0,Session id,5543
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(31480, 82)"
5,Transformed train set shape,"(22036, 82)"
6,Transformed test set shape,"(9444, 82)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [20]:
result = test_and_save_pre_processing_approach('generic_pre_processing', generic_pre_processing_setup)
models_comparison_generic = result[0]
dataset_generic_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9999,1.0000,0.9000,1.0000,0.9333,0.9333,0.9414,1.9740
et,Extra Trees Classifier,0.9998,1.0000,0.8167,1.0000,0.8800,0.8799,0.8937,0.6660
xgboost,Extreme Gradient Boosting,0.9998,0.9999,0.7833,0.9000,0.8267,0.8266,0.8340,0.7550
rf,Random Forest Classifier,0.9996,0.9997,0.6333,0.9000,0.7167,0.7166,0.7405,1.1570
dt,Decision Tree Classifier,0.9994,0.9164,0.8333,0.7000,0.7505,0.7502,0.7581,0.4220
knn,K Neighbors Classifier,0.9992,0.8998,0.4500,0.7000,0.5067,0.5064,0.5389,0.8030
gbc,Gradient Boosting Classifier,0.9991,0.8809,0.6500,0.6750,0.6390,0.6386,0.6501,8.8270
ridge,Ridge Classifier,0.9990,0.9571,0.2333,0.3667,0.2633,0.2632,0.2807,0.2630
qda,Quadratic Discriminant Analysis,0.9989,0.8380,0.0000,0.0000,0.0000,0.0000,0.0000,0.4090
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1530


### 2.2 Specifyng date feature pre-processing
- Discarding year
- Adding second, day and month features

This discarding is necessary beacause the year of the network package must not be taken as a variable to determine if the package is malicious or not

In [24]:
date_specific_pre_processing_setup = setup(
                                    dataset, 
                                    target = 'Label',
                                    date_features = ['Timestamp'],
                                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                    imputation_type = 'simple',
                                    numeric_imputation = 'mean',
                                    categorical_imputation = 'mode',
                                    fold_strategy = 'stratifiedkfold',
                                    fold = 10
                                )

,Description,Value
0,Session id,5991
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(31480, 84)"
5,Transformed train set shape,"(22036, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [25]:
result = test_and_save_pre_processing_approach('date_specific_pre_processing', date_specific_pre_processing_setup)
models_comparison_date_specific = result[0]
dataset_date_specific_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9667,1.0000,0.9800,0.9800,0.9816,1.9710
xgboost,Extreme Gradient Boosting,0.9998,0.9997,0.7833,1.0000,0.8600,0.8599,0.8753,0.7120
et,Extra Trees Classifier,0.9996,0.9999,0.6333,0.9000,0.7233,0.7232,0.7440,0.5140
rf,Random Forest Classifier,0.9995,0.9998,0.5000,0.8000,0.5933,0.5932,0.6201,0.9830
dt,Decision Tree Classifier,0.9993,0.8248,0.6500,0.7567,0.6571,0.6568,0.6792,0.3300
knn,K Neighbors Classifier,0.9992,0.8665,0.3833,0.4333,0.4000,0.3999,0.4039,0.7110
ridge,Ridge Classifier,0.9992,0.9383,0.3500,0.5000,0.3967,0.3965,0.4100,0.1710
gbc,Gradient Boosting Classifier,0.9992,0.8747,0.6167,0.6500,0.6033,0.6030,0.6177,8.9460
lr,Logistic Regression,0.9991,0.9285,0.3500,0.4000,0.3667,0.3664,0.3705,3.8140
qda,Quadratic Discriminant Analysis,0.9989,0.8598,0.0000,0.0000,0.0000,0.0000,0.0000,0.4250


### 2.3 Missing values pre-processing

In [11]:
simple_imputation_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'simple',
                                                numeric_imputation = 'knn',
                                                categorical_imputation = 'mode',
                                                fold = 10
                                                )

,Description,Value
0,Session id,3903
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(31480, 84)"
5,Transformed train set shape,"(22036, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [12]:
result = test_and_save_pre_processing_approach('simple_imputation_pre_processing', simple_imputation_pre_processing_setup)
models_comparison_simple_imputation = result[0]
dataset_simple_imputation_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9998,0.9920,0.8667,0.9667,0.8933,0.8932,0.9047,3.3090
et,Extra Trees Classifier,0.9997,1.0000,0.7667,0.9000,0.8000,0.7999,0.8154,0.7910
xgboost,Extreme Gradient Boosting,0.9997,0.9999,0.7500,1.0000,0.8300,0.8299,0.8514,0.7910
rf,Random Forest Classifier,0.9995,0.9998,0.5667,0.9000,0.6667,0.6665,0.6982,1.1190
dt,Decision Tree Classifier,0.9994,0.8582,0.7167,0.6767,0.6783,0.6780,0.6870,0.5810
gbc,Gradient Boosting Classifier,0.9991,0.9201,0.6167,0.6433,0.6017,0.6012,0.6151,7.2400
lightgbm,Light Gradient Boosting Machine,0.9987,0.8995,0.7333,0.5595,0.6015,0.6010,0.6220,0.9880


In [26]:
iterative_imputation_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'iterative',
                                                fold_strategy = 'stratifiedkfold',
                                                fold = 10
                                                )

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005555 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12901
[LightGBM] [Info] Number of data points in the train set: 21925, number of used features: 70
[LightGBM] [Info] Start training from score 301342.783134
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12901
[LightGBM] [Info] Number of data points in the train set: 21925, number of used features: 70
[LightGBM] [Info] Start training from score 28209.183952
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015280 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Inf

,Description,Value
0,Session id,3091
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(31480, 84)"
5,Transformed train set shape,"(22036, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [27]:
result = test_and_save_pre_processing_approach('iterative_imputation_pre_processing', iterative_imputation_pre_processing_setup)
models_comparison_iterative_imputation = result[0]
dataset_iterative_imputation_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9500,1.0000,0.9667,0.9666,0.9707,7.7250
et,Extra Trees Classifier,0.9997,1.0000,0.7500,1.0000,0.8300,0.8299,0.8514,7.2450
xgboost,Extreme Gradient Boosting,0.9997,0.9998,0.7000,1.0000,0.7967,0.7965,0.8221,7.1850
rf,Random Forest Classifier,0.9995,0.9996,0.5500,1.0000,0.6933,0.6931,0.7322,5.3060
gbc,Gradient Boosting Classifier,0.9995,0.9464,0.7500,0.8000,0.7433,0.7431,0.7582,21.5450
dt,Decision Tree Classifier,0.9994,0.8998,0.8000,0.7900,0.7505,0.7502,0.7718,4.9050
knn,K Neighbors Classifier,0.9993,0.8832,0.3333,0.7000,0.4467,0.4465,0.4798,6.0180
ridge,Ridge Classifier,0.9993,0.9762,0.3333,0.7000,0.4467,0.4465,0.4798,4.3980
lr,Logistic Regression,0.9990,0.9056,0.3333,0.4000,0.3305,0.3301,0.3474,10.1000
qda,Quadratic Discriminant Analysis,0.9989,0.8746,0.0000,0.0000,0.0000,0.0000,0.0000,4.6400


#### Conclusion
Analyzing mainly the Recall and F1 metrics it was possible do see:
- The Decision Tree and Extra Trees algorithms were improved
- The Ada Boost and Extreme Gradiend Boosting were deteriorated

### 2.4 Fix imbalance pre-processing

In [34]:
fix_imbalance_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'simple',
                                                numeric_imputation = 'mean',
                                                categorical_imputation = 'mode',
                                                fold_strategy = 'stratifiedkfold',
                                                fold = 10,
                                                fix_imbalance = True,
                                                fix_imbalance_method = 'SMOTE'
                                                )

,Description,Value
0,Session id,476
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [35]:
result = test_and_save_pre_processing_approach('fix_imbalance_pre_processing', fix_imbalance_pre_processing_setup)
models_comparison_fix_imbalance = result[0]
dataset_fix_imbalance_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9667,1.0000,0.9800,0.9800,0.9816,5.2210
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,20.0130
xgboost,Extreme Gradient Boosting,1.0000,1.0000,0.9500,1.0000,0.9667,0.9666,0.9707,1.0170
et,Extra Trees Classifier,0.9998,1.0000,0.7833,0.9000,0.8167,0.8166,0.8284,1.4800
lightgbm,Light Gradient Boosting Machine,0.9998,0.9648,0.8667,0.9667,0.8933,0.8932,0.9047,1.3690
rf,Random Forest Classifier,0.9997,1.0000,0.7500,1.0000,0.8300,0.8299,0.8514,3.2000
dt,Decision Tree Classifier,0.9996,0.8500,0.7000,0.8250,0.7324,0.7322,0.7466,0.9970
qda,Quadratic Discriminant Analysis,0.9992,0.9077,0.3333,0.6000,0.4167,0.4166,0.4405,0.6660
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2410
knn,K Neighbors Classifier,0.9961,0.9406,0.8833,0.2160,0.3403,0.3391,0.4289,1.5180


#### Conclusion
Analyzing mainly the Recall and F1 metrics it was possible do see:
- The SMOTE method proved be the most suitable
- The Ada, xgboost, gbc and rf were improved
- The Decision Tree were deteriorated

### 2.5 Remove Outliers pre-processing

In [39]:
remove_outliers_pre_processing_setup = setup(
                                            dataset, 
                                            target = 'Label',
                                            date_features = ['Timestamp'],
                                            create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                            imputation_type = 'simple',
                                            numeric_imputation = 'mean',
                                            categorical_imputation = 'mode',
                                            fold_strategy = 'stratifiedkfold',
                                            fold = 10,
                                            fix_imbalance = True,
                                            fix_imbalance_method = 'SMOTE',
                                            remove_outliers = True,
                                            outliers_method = 'iforest'
                                            )

,Description,Value
0,Session id,7987
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(51274, 84)"
5,Transformed train set shape,"(41830, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [40]:
result = test_and_save_pre_processing_approach('remove_outliers_pre_processing', remove_outliers_pre_processing_setup)
models_comparison_remove_outliers = result[0]
dataset_remove_outliers_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9997,0.9993,0.8000,0.9167,0.8267,0.8265,0.8416,1.5010
dt,Decision Tree Classifier,0.9996,0.8749,0.7500,0.9000,0.7833,0.7831,0.8031,0.7530
ada,Ada Boost Classifier,0.9996,0.9806,0.7167,0.9333,0.7833,0.7831,0.8031,4.5610
gbc,Gradient Boosting Classifier,0.9996,0.9997,0.7167,0.9500,0.8000,0.7998,0.8161,17.9570
et,Extra Trees Classifier,0.9996,0.9985,0.6667,0.8500,0.7267,0.7265,0.7416,1.7210
rf,Random Forest Classifier,0.9995,0.9992,0.5833,0.8000,0.6567,0.6565,0.6733,2.9030
lightgbm,Light Gradient Boosting Machine,0.9995,0.8931,0.7167,0.8833,0.7667,0.7665,0.7824,1.6730
qda,Quadratic Discriminant Analysis,0.9993,0.7922,0.4000,0.9000,0.5500,0.5498,0.5973,0.8490
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5150
knn,K Neighbors Classifier,0.9959,0.8571,0.7167,0.1794,0.2803,0.2790,0.3507,1.4140


In [42]:
remove_outliers_2_pre_processing_setup = setup(
                                            dataset, 
                                            target = 'Label',
                                            date_features = ['Timestamp'],
                                            create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                            imputation_type = 'simple',
                                            numeric_imputation = 'mean',
                                            categorical_imputation = 'mode',
                                            fold_strategy = 'stratifiedkfold',
                                            fold = 10,
                                            fix_imbalance = True,
                                            fix_imbalance_method = 'SMOTE',
                                            remove_outliers = True,
                                            outliers_method = 'ee'
                                            )

,Description,Value
0,Session id,7027
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(51264, 84)"
5,Transformed train set shape,"(41820, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [43]:
result = test_and_save_pre_processing_approach('remove_outliers_2_pre_processing', remove_outliers_2_pre_processing_setup)
models_comparison_remove_outliers_2 = result[0]
dataset_remove_outliers_2_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9500,1.0000,0.9667,0.9666,0.9707,7.6460
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,20.5610
rf,Random Forest Classifier,0.9998,0.9997,0.7833,1.0000,0.8500,0.8499,0.8698,6.9840
et,Extra Trees Classifier,0.9998,1.0000,0.9000,0.9750,0.9157,0.9156,0.9259,4.2650
dt,Decision Tree Classifier,0.9997,0.9166,0.8333,0.8750,0.8424,0.8422,0.8480,4.5150
xgboost,Extreme Gradient Boosting,0.9997,0.9995,0.9333,0.8667,0.8933,0.8932,0.8965,4.2970
qda,Quadratic Discriminant Analysis,0.9992,0.9518,0.2500,0.4000,0.2967,0.2966,0.3100,4.2490
lightgbm,Light Gradient Boosting Machine,0.9992,0.9993,0.8833,0.7461,0.7940,0.7937,0.8037,4.9030
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,3.7910
knn,K Neighbors Classifier,0.9958,0.9574,0.9167,0.2023,0.3290,0.3278,0.4270,4.4960


In [44]:
remove_outliers_3_pre_processing_setup = setup(
                                            dataset, 
                                            target = 'Label',
                                            date_features = ['Timestamp'],
                                            create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                            imputation_type = 'simple',
                                            numeric_imputation = 'mean',
                                            categorical_imputation = 'mode',
                                            fold_strategy = 'stratifiedkfold',
                                            fold = 10,
                                            fix_imbalance = True,
                                            fix_imbalance_method = 'SMOTE',
                                            remove_outliers = True,
                                            outliers_method = 'lof'
                                            )

,Description,Value
0,Session id,288
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(51286, 84)"
5,Transformed train set shape,"(41842, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [45]:
result = test_and_save_pre_processing_approach('remove_outliers_3_pre_processing', remove_outliers_3_pre_processing_setup)
models_comparison_remove_outliers_3 = result[0]
dataset_remove_outliers_3_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9996,0.9999,0.6667,0.9667,0.7567,0.7565,0.7854,22.7980
dt,Decision Tree Classifier,0.9995,0.7916,0.5833,0.8250,0.6624,0.6622,0.6826,2.6170
rf,Random Forest Classifier,0.9995,0.9048,0.5333,0.8000,0.6267,0.6265,0.6460,4.9970
ada,Ada Boost Classifier,0.9995,0.9460,0.5833,0.8750,0.6790,0.6789,0.7033,6.6810
et,Extra Trees Classifier,0.9995,0.9986,0.4833,0.7000,0.5600,0.5599,0.5753,3.3990
xgboost,Extreme Gradient Boosting,0.9995,0.9857,0.5833,0.8250,0.6624,0.6622,0.6826,3.3930
lightgbm,Light Gradient Boosting Machine,0.9994,0.7925,0.5833,0.7750,0.6457,0.6455,0.6618,3.7460
qda,Quadratic Discriminant Analysis,0.9990,0.6992,0.1500,0.3000,0.2000,0.1999,0.2121,2.6280
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,2.2980
knn,K Neighbors Classifier,0.9945,0.7645,0.5333,0.1021,0.1696,0.1682,0.2296,3.2470


#### Conclusion
Analyzing mainly the Recall and F1 metrics it was possible do see:
- The ee method proved be the most suitable
- The only algortihm that has been improved was the Extra trees
- All the others algorithms has been deteriorated

So we wil not use remove_outliers method

### 2.6 Normalize pre-processing

In [46]:
normalize_pre_processing_setup = setup(
                                    dataset, 
                                    target = 'Label',
                                    date_features = ['Timestamp'],
                                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                    imputation_type = 'simple',
                                    numeric_imputation = 'mean',
                                    categorical_imputation = 'mode',
                                    fold_strategy = 'stratifiedkfold',
                                    fold = 10,
                                    fix_imbalance = True,
                                    fix_imbalance_method = 'SMOTE',
                                    normalize = True,
                                    # Test different zscore
                                    normalize_method = 'zscore'
                                    )

,Description,Value
0,Session id,1744
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [47]:
result = test_and_save_pre_processing_approach('normalize_pre_processing', normalize_pre_processing_setup)
models_comparison_normalize = result[0]
dataset_normalize_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9667,1.0000,0.9800,0.9800,0.9816,4.5270
gbc,Gradient Boosting Classifier,0.9999,1.0000,1.0000,0.9333,0.9600,0.9600,0.9633,17.2730
xgboost,Extreme Gradient Boosting,0.9999,0.9999,0.9167,0.9667,0.9333,0.9333,0.9373,1.1490
rf,Random Forest Classifier,0.9998,0.9997,0.8333,0.9500,0.8667,0.8666,0.8784,2.9210
et,Extra Trees Classifier,0.9998,0.9999,0.7833,1.0000,0.8500,0.8499,0.8698,1.4300
lightgbm,Light Gradient Boosting Machine,0.9997,0.9937,0.8667,0.8167,0.8333,0.8332,0.8373,1.5870
dt,Decision Tree Classifier,0.9994,0.8582,0.7167,0.7333,0.7000,0.6997,0.7120,0.9780
qda,Quadratic Discriminant Analysis,0.9992,0.9407,0.3000,0.6000,0.3833,0.3832,0.4145,1.1700
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3210
knn,K Neighbors Classifier,0.9930,0.8220,0.6500,0.0947,0.1645,0.1629,0.2449,1.3350


In [48]:
normalize_2_pre_processing_setup = setup(
                                    dataset, 
                                    target = 'Label',
                                    date_features = ['Timestamp'],
                                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                    imputation_type = 'simple',
                                    numeric_imputation = 'mean',
                                    categorical_imputation = 'mode',
                                    fold_strategy = 'stratifiedkfold',
                                    fold = 10,
                                    fix_imbalance = True,
                                    fix_imbalance_method = 'SMOTE',
                                    normalize = True,
                                    # Test different zscore
                                    normalize_method = 'minmax'
                                    )

,Description,Value
0,Session id,4303
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [49]:
result = test_and_save_pre_processing_approach('normalize_2_pre_processing', normalize_2_pre_processing_setup)
models_comparison_normalize_2 = result[0]
dataset_normalize_2_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9999,0.9990,0.8500,0.9000,0.8667,0.8666,0.8707,2.1720
ada,Ada Boost Classifier,0.9999,0.9989,0.9167,0.9500,0.9300,0.9299,0.9316,3.6970
gbc,Gradient Boosting Classifier,0.9999,0.9999,0.9500,0.9167,0.9300,0.9299,0.9316,18.6650
dt,Decision Tree Classifier,0.9998,0.9583,0.9167,0.9267,0.9017,0.9016,0.9114,0.7220
et,Extra Trees Classifier,0.9998,0.9999,0.8000,0.9000,0.8333,0.8333,0.8414,1.6800
xgboost,Extreme Gradient Boosting,0.9998,0.9996,0.9167,0.9333,0.9067,0.9066,0.9156,1.2860
lightgbm,Light Gradient Boosting Machine,0.9996,0.9649,0.8667,0.8517,0.8440,0.8439,0.8513,2.1000
qda,Quadratic Discriminant Analysis,0.9992,0.9524,0.2667,0.6000,0.3667,0.3665,0.3982,0.6890
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3240
knn,K Neighbors Classifier,0.9915,0.8972,0.7333,0.0982,0.1674,0.1659,0.2578,1.1370


In [50]:
normalize_3_pre_processing_setup = setup(
                                    dataset, 
                                    target = 'Label',
                                    date_features = ['Timestamp'],
                                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                    imputation_type = 'simple',
                                    numeric_imputation = 'mean',
                                    categorical_imputation = 'mode',
                                    fold_strategy = 'stratifiedkfold',
                                    fold = 10,
                                    fix_imbalance = True,
                                    fix_imbalance_method = 'SMOTE',
                                    normalize = True,
                                    # Test different zscore
                                    normalize_method = 'maxabs'
                                    )

,Description,Value
0,Session id,1648
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [51]:
result = test_and_save_pre_processing_approach('normalize_3_pre_processing', normalize_3_pre_processing_setup)
models_comparison_normalize_3 = result[0]
dataset_normalize_3_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,1.0000,1.0000,1.0000,0.9667,0.9800,0.9800,0.9816,1.0130
rf,Random Forest Classifier,0.9999,0.9999,0.9167,1.0000,0.9467,0.9466,0.9523,2.9710
ada,Ada Boost Classifier,0.9999,1.0000,1.0000,0.9417,0.9657,0.9657,0.9682,4.6390
gbc,Gradient Boosting Classifier,0.9998,1.0000,1.0000,0.8833,0.9267,0.9266,0.9339,17.9060
et,Extra Trees Classifier,0.9998,1.0000,0.8333,1.0000,0.8933,0.8932,0.9047,1.2760
lightgbm,Light Gradient Boosting Machine,0.9998,1.0000,0.9167,0.8833,0.8900,0.8899,0.8948,1.2770
dt,Decision Tree Classifier,0.9997,0.9416,0.8833,0.8833,0.8667,0.8665,0.8747,1.0870
qda,Quadratic Discriminant Analysis,0.9990,0.9519,0.0500,0.1000,0.0667,0.0666,0.0707,0.7720
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3500
knn,K Neighbors Classifier,0.9949,0.9143,0.7333,0.1559,0.2542,0.2528,0.3333,2.0490


In [52]:
normalize_4_pre_processing_setup = setup(
                                    dataset, 
                                    target = 'Label',
                                    date_features = ['Timestamp'],
                                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                    imputation_type = 'simple',
                                    numeric_imputation = 'mean',
                                    categorical_imputation = 'mode',
                                    fold_strategy = 'stratifiedkfold',
                                    fold = 10,
                                    fix_imbalance = True,
                                    fix_imbalance_method = 'SMOTE',
                                    normalize = True,
                                    # Test different zscore
                                    normalize_method = 'robust'
                                    )

,Description,Value
0,Session id,4117
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [53]:
result = test_and_save_pre_processing_approach('normalize_4_pre_processing', normalize_4_pre_processing_setup)
models_comparison_normalize_4 = result[0]
dataset_normalize_4_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9999,1.0000,0.9500,0.9667,0.9467,0.9466,0.9523,3.9800
et,Extra Trees Classifier,0.9999,1.0000,0.9167,1.0000,0.9467,0.9466,0.9523,1.3030
gbc,Gradient Boosting Classifier,0.9998,1.0000,1.0000,0.9067,0.9371,0.9371,0.9448,16.6160
xgboost,Extreme Gradient Boosting,0.9998,0.9999,0.9000,0.9083,0.8790,0.8789,0.8912,1.0010
rf,Random Forest Classifier,0.9997,1.0000,0.6833,0.9000,0.7500,0.7499,0.7698,2.7930
dt,Decision Tree Classifier,0.9996,0.9166,0.8333,0.8917,0.8257,0.8255,0.8435,0.9370
lightgbm,Light Gradient Boosting Machine,0.9996,0.9998,0.9000,0.8317,0.8395,0.8394,0.8521,1.2880
qda,Quadratic Discriminant Analysis,0.9992,0.9454,0.2500,0.4000,0.2967,0.2966,0.3100,0.6160
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3050
knn,K Neighbors Classifier,0.9955,0.8902,0.6833,0.1503,0.2400,0.2387,0.3131,1.1880


#### Conclusion

Analyzing mainly the Recall and F1 Score it was possible to see:

- The maxabs was the most suitable normalize method
- This normalization generally improved the Recall and F1 Score of the main algorithms
- The only exceptions were the F1 Score of the ada and gbc algorithms, that slightly decreased 

So this method could be a good choice for pre-processing


### 2.7 Feature Transform pre-processing

In [55]:
transformation_feature_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'simple',
                                                numeric_imputation = 'mean',
                                                categorical_imputation = 'mode',
                                                fold_strategy = 'stratifiedkfold',
                                                fold = 10,
                                                fix_imbalance = True,
                                                fix_imbalance_method = 'SMOTE',
                                                transformation = True,
                                                transformation_method = 'yeo-johnson'
                                                )

,Description,Value
0,Session id,4119
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [56]:
result = test_and_save_pre_processing_approach('transformation_feature_pre_processing', transformation_feature_pre_processing_setup)
models_comparison_transformation_feature = result[0]
dataset_transformation_feature_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9999,0.9997,0.9667,0.9667,0.9600,0.9600,0.9633,7.5950
gbc,Gradient Boosting Classifier,0.9999,1.0000,1.0000,0.9000,0.9400,0.9399,0.9449,23.8420
xgboost,Extreme Gradient Boosting,0.9999,0.9999,0.9500,0.9500,0.9500,0.9500,0.9500,3.5020
qda,Quadratic Discriminant Analysis,0.9998,0.9933,0.8333,0.9000,0.8600,0.8600,0.8633,2.4360
et,Extra Trees Classifier,0.9998,1.0000,0.7833,1.0000,0.8600,0.8599,0.8753,3.4610
rf,Random Forest Classifier,0.9997,1.0000,0.7500,1.0000,0.8400,0.8399,0.8570,4.5210
lightgbm,Light Gradient Boosting Machine,0.9997,0.9639,0.8167,0.8500,0.8181,0.8179,0.8254,3.7770
dt,Decision Tree Classifier,0.9996,0.9249,0.8500,0.8000,0.8214,0.8212,0.8230,2.1580
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,2.1320
knn,K Neighbors Classifier,0.9986,0.9081,0.8167,0.4308,0.5512,0.5506,0.5850,2.7290


In [57]:
transformation_feature_2_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'simple',
                                                numeric_imputation = 'mean',
                                                categorical_imputation = 'mode',
                                                fold_strategy = 'stratifiedkfold',
                                                fold = 10,
                                                fix_imbalance = True,
                                                fix_imbalance_method = 'SMOTE',
                                                transformation = True,
                                                transformation_method = 'quantile'
                                                )

,Description,Value
0,Session id,3524
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 84)"
5,Transformed train set shape,"(44024, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [58]:
result = test_and_save_pre_processing_approach('transformation_feature_2_pre_processing', transformation_feature_2_pre_processing_setup)
models_comparison_transformation_feature_2 = result[0]
dataset_transformation_feature_2_pre_processing = result[1]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,1.0000,1.0000,0.9667,1.0000,0.9800,0.9800,0.9816,5.4890
gbc,Gradient Boosting Classifier,0.9999,1.0000,0.9667,0.9500,0.9467,0.9466,0.9523,23.8660
rf,Random Forest Classifier,0.9998,0.9999,0.8500,1.0000,0.9067,0.9066,0.9156,3.5640
xgboost,Extreme Gradient Boosting,0.9998,1.0000,0.9000,0.9667,0.9267,0.9266,0.9299,1.4570
lightgbm,Light Gradient Boosting Machine,0.9998,0.9888,0.9167,0.9417,0.9190,0.9190,0.9239,2.0020
et,Extra Trees Classifier,0.9997,0.9990,0.8333,0.9333,0.8600,0.8599,0.8713,1.6060
dt,Decision Tree Classifier,0.9996,0.9166,0.8333,0.8750,0.8257,0.8255,0.8395,1.1450
dummy,Dummy Classifier,0.9989,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.6590
knn,K Neighbors Classifier,0.9985,0.8745,0.7000,0.4100,0.5011,0.5005,0.5262,1.5430
lr,Logistic Regression,0.9915,0.9932,0.8500,0.1242,0.2035,0.2020,0.3056,2.3510


#### Conclusion

Analyzing mainly the Recall and F1 Score it was possible to see:

- This method brign only deterioration to the mainly algortithms that we are considering

So we will not use the Transformation feature method

### 2.8 Polynomial Features pre-processing

In [59]:
polynomial_features_pre_processing_setup = setup(
                                                dataset, 
                                                target = 'Label',
                                                date_features = ['Timestamp'],
                                                create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                                                imputation_type = 'simple',
                                                numeric_imputation = 'mean',
                                                categorical_imputation = 'mode',
                                                fold_strategy = 'stratifiedkfold',
                                                fold = 10,
                                                fix_imbalance = True,
                                                fix_imbalance_method = 'SMOTE',
                                                normalize = True,
                                                normalize_method = 'maxabs',
                                                polynomial_features = True
                                                )

,Description,Value
0,Session id,1726
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(53468, 3570)"
5,Transformed train set shape,"(44024, 3570)"
6,Transformed test set shape,"(9444, 3570)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [62]:
result = test_and_save_pre_processing_approach('polynomial_features_pre_processing', polynomial_features_pre_processing_setup)
models_comparison_polynomial_features = result[0]
dataset_polynomial_features_pre_processing = result[1]

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:57:42
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Ada Boost Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/33 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
my_setup = setup(
                    dataset, 
                    target = 'Label',
                    categorical_imputation= 'drop',
                    date_features = ['Timestamp'],
                    create_date_columns = ['hour', 'minute', 'second', 'day', 'month'],
                    remove_outliers = True,
                    normalize = True,
                    transformation= True
                    )

,Description,Value
0,Session id,4391
1,Target,Label
2,Target type,Binary
3,Original data shape,"(31480, 80)"
4,Transformed data shape,"(30378, 84)"
5,Transformed train set shape,"(20934, 84)"
6,Transformed test set shape,"(9444, 84)"
7,Numeric features,78
8,Date features,1
9,Rows with missing values,0.5%


In [9]:
dataset_pre_processed = get_config('dataset_transformed')


In [10]:
dataset_pre_processed.head()

,Dst Port,Protocol,Timestamp_month,Timestamp_day,Timestamp_second,Timestamp_minute,Timestamp_hour,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
10086,-0.875631,1.406264,0.0,0.0,0.863434,-0.102546,0.980888,-0.973095,-1.236807,-0.204330,0.147012,0.151641,0.247316,1.541111,0.627952,-0.781132,0.192203,1.516131,0.416272,-0.698808,1.340328,0.974290,-0.859120,-0.915938,-0.939333,0.244360,-1.244248,-1.254543,-0.806812,-1.249574,-1.172718,-0.768606,-0.769539,-0.694583,-0.769579,-0.693568,-0.19343,0.0,0.0,0.0,-1.190932,-0.230214,0.962451,1.391014,1.544722,0.070441,0.433691,-0.319140,-0.345151,-0.077816,-0.19343,-0.375339,-0.735580,-0.695283,-0.213302,0.0,-0.375339,1.091241,0.566859,0.627952,0.416272,0.0,0.0,0.0,0.0,0.0,0.0,-1.236807,0.147012,-0.204330,0.151641,-1.277618,-0.803023,-0.844750,-1.406042,-0.253069,-0.212689,-0.253069,-0.253069,-0.419368,-0.240185,-0.419368,-0.419368,0
30615,0.030027,-0.548838,0.0,0.0,0.358858,-1.600531,-1.587104,-1.219234,-1.236807,0.301853,-1.294371,-1.186369,-1.302111,-0.664841,-1.348430,-0.781132,-1.194271,-0.648506,-1.223592,-0.698808,-1.258318,1.325888,-1.348909,-0.339868,-1.318863,-0.359314,-1.244248,-1.254543,-0.806812,-1.249574,-1.172718,0.199512,0.292813,-0.694583,0.241704,0.865372,-0.19343,0.0,0.0,0.0,-0.562648,0.525507,1.216049,1.606536,-0.662541,-1.310286,-1.343990,-1.261251,-1.255271,12.850792,-0.19343,-0.375339,-0.735580,1.438263,-0.213302,0.0,-0.375339,1.795683,-1.359465,-1.348430,-1.223592,0.0,0.0,0.0,0.0,0.0,0.0,-1.236807,-1.294371,0.301853,-1.186369,-1.085851,-0.297542,-0.844750,0.655312,-0.253069,-0.212689,-0.253069,-0.253069,-0.419368,-0.240185,-0.419368,-0.419368,0
7573,0.030027,-0.548838,0.0,0.0,-0.356932,0.893848,0.980888,0.078511,1.257074,1.343627,1.052833,1.806480,0.827502,-0.664841,0.761665,1.264898,1.409982,-0.648506,2.066370,1.521344,1.081562,0.186143,-0.220381,0.794364,-0.018013,-1.538304,0.341854,0.205626,1.085462,0.263187,-0.237991,1.154240,1.086528,1.336146,1.180924,-0.270723,-0.19343,0.0,0.0,0.0,1.134869,1.249620,0.174517,0.781669,-0.662541,1.456828,2.222260,1.708773,1.701890,-0.077816,-0.19343,-0.375339,1.359472,-0.695283,-0.213302,0.0,-0.375339,1.091241,2.197314,0.761665,2.066370,0.0,0.0,0.0,0.0,0.0,0.0,1.257074,1.052833,1.343627,1.806480,0.823387,0.714746,1.335903,0.655312,-0.253069,-0.212689,-0.253069,-0.253069,-0.419368,-0.240185,-0.419368,-0.419368,0
15416,-0.875631,1.406264,0.0,0.0,-1.343991,0.120308,0.476759,-0.670184,-1.236807,-0.204330,0.060633,0.095777,0.149924,1.516577,0.454266,-0.781132,0.130588,1.496691,0.329538,-0.698808,0.978157,0.621181,-0.485872,-0.915938,-0.615312,0.599075,-1.244248,-1.254543,-0.806812,-1.249574,-1.172718,-0.768606,-0.769539,-0.694583,-0.769579,-0.693568,-0.19343,0.0,0.0,0.0,-1.190932,-0.230214,0.617584,1.152790,1.520521,0.004498,0.300963,-0.319140,-0.345151,-0.077816,-0.19343,-0.375339,-0.735580,-0.695283,-0.213302,0.0,-0.375339,1.091241,0.420519,0.454266,0.329538,0.0,0.0,0.0,0.0,0.0,0.0,-1.236807,0.060633,-0.204330,0.095777,-1.277618,-0.803023,-0

In [11]:
file_path = file_path = os.path.join(os.getcwd(), '..', 'Datasets', 'CSE-CIC-IDS2018', 'pre-processed','0_01-dataset-pre-processed-by-pycaret.csv')
dataset_pre_processed.to_csv(file_path, index=False)

In [12]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9996,0.9641,0.6333,0.9000,0.7267,0.7265,0.7460,2.6390
xgboost,Extreme Gradient Boosting,0.9995,0.9986,0.4667,0.7000,0.5433,0.5432,0.5624,1.3240
dt,Decision Tree Classifier,0.9994,0.7999,0.6000,0.6583,0.6157,0.6155,0.6222,1.2740
gbc,Gradient Boosting Classifier,0.9994,0.9001,0.6500,0.7333,0.6733,0.6731,0.6821,6.7780
et,Extra Trees Classifier,0.9993,0.9986,0.3667,0.6000,0.4333,0.4332,0.4568,1.3200
rf,Random Forest Classifier,0.9992,0.9736,0.2833,0.5000,0.3467,0.3466,0.3677,1.6470
lr,Logistic Regression,0.9990,0.9829,0.1167,0.3000,0.1667,0.1666,0.1861,2.1730
knn,K Neighbors Classifier,0.9990,0.7165,0.0500,0.1000,0.0667,0.0666,0.0707,1.4570
svm,SVM - Linear Kernel,0.9989,0.5906,0.0000,0.0000,0.0000,0.0000,0.0000,1.1760
ridge,Ridge Classifier,0.9989,0.9913,0.0000,0.0000,0.0000,0.0000,0.0000,1.0660
